In [1]:
import torch
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader
import cv2 as cv
import os

import matplotlib.pyplot as plt
import seaborn as sns
from umap import UMAP
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
import numpy as np


In [2]:

fe = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')

# data = torch.load('checkpoints/sber_objects_features_augs.pth')
data = torch.load('m_platform_objects_10aug.pth')
features, labels = data['x'], data['y']

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.5, random_state=1)

# model = LogisticRegression()
# # model = KNeighborsClassifier(weights='distance', metric='cosine')

# model.fit(x_train, y_train)

# print(accuracy_score(y_test, model.predict(x_test)))


Using cache found in /home/iiwa/.cache/torch/hub/facebookresearch_dino_main


In [3]:
classes = list(set(labels))

known_classes, novel_classes = classes[:int(len(classes) * 0.7)], classes[int(len(classes) * 0.7):]


In [4]:

train_indxs = []
labels = np.array(labels)
for c in known_classes:
    train_indxs += list(np.where(labels == c)[0])

known_features = features[train_indxs]

test_indxs = []
for c in novel_classes:
    test_indxs += list(np.where(labels == c)[0])

novel_features = features[test_indxs]


In [5]:
# train_features = features[labels in train_classes]
known_features.shape, novel_features.shape

(torch.Size([160, 384]), torch.Size([120, 384]))

In [23]:
from sklearn.metrics import accuracy_score
from sklearn.svm import OneClassSVM
from sklearn.linear_model import SGDOneClassSVM


train_features, test_known_features = train_test_split(known_features, test_size=0.2, random_state=1)

# model = LocalOutlierFactor(n_neighbors=20, metric='cosine', novelty=True)
model = OneClassSVM(nu=0.1)
# model = SGDOneClassSVM(nu=0.9)

model.fit(train_features)

test_features = np.concatenate([test_known_features, novel_features])


gts = np.concatenate([np.ones((len(test_known_features))), -1 * np.ones((len(novel_features)))])

accuracy_score(model.predict(test_known_features), np.ones((len(test_known_features)))), \
    accuracy_score(model.predict(novel_features), -1 * np.ones((len(novel_features))))

(0.875, 1.0)

In [40]:
test_known_features.shape

torch.Size([32, 384])

In [25]:
train_features = features[train_indxs]
test_features = features[test_indxs]

train_labels = labels[train_indxs]
test_labels = labels[test_indxs]

In [26]:
model = LocalOutlierFactor(n_neighbors=10, distance='weights', novelty=True)
model.fit()

torch.Size([69, 384])

SyntaxError: invalid syntax (3490902136.py, line 3)